In [1]:
import chess
from stockfish import Stockfish
stockfish = Stockfish(path=r"C:\Users\athar\Chess engine using imitation learning\stockfish\stockfish-windows-x86-64-avx2.exe")
import random
from pprint import pprint
import numpy as np
import os
import glob
import time

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [37]:
#test checkmate:
board = chess.Board()
board.push_san("e4")
board.push_san("e5")
board.push_san("Qh5")
board.push_san("Nc6")
board.push_san("Bc4")
board.push_san("Nf6")
board.push_san("Qxf7")
board.is_checkmate()
# board

True

In [38]:
stockfish.get_best_move()


'e2e4'

In [39]:
def fen_to_board(fen):
    board = []
    for row in fen.split('/'):
        brow = []
        for c in row:
            if c == ' ':
                break
            elif c in '12345678':
                brow.extend( ['--'] * int(c) )
            elif c == 'p':
                brow.append( 'bp' )
            elif c == 'P':
                brow.append( 'wp' )
            elif c > 'Z':
                brow.append( 'b'+c.upper() )
            else:
                brow.append( 'w'+c )

        board.append( brow )
    return board

In [16]:
#helper functions:
def checkEndCondition(board):
 if (board.is_checkmate() or board.is_stalemate() or board.is_insufficient_material() or board.can_claim_threefold_repetition() or board.can_claim_fifty_moves() or board.can_claim_draw()):
  return True
 return False

#save
def findNextIdx():
 files = (glob.glob(r"C:\Users\athar\Chess engine using imitation learning\data\rawData\*.npy"))
 if (len(files) == 0):
  return 1 #if no files, return 1
 highestIdx = 0
 for f in files:
  file = f
  currIdx = file.split("movesAndPositions")[-1].split(".npy")[0]
  highestIdx = max(highestIdx, int(currIdx))

 return int(highestIdx)+1

def saveData(moves, positions):
 moves = np.array(moves).reshape(-1, 1)
 positions = np.array(positions).reshape(-1,1)
 movesAndPositions = np.concatenate((moves, positions), axis = 1)
 nextIdx = findNextIdx()
 np.save(f"data/rawData/movesAndPositions{nextIdx}.npy", movesAndPositions)
 print("Saved successfully")

def runGame(numMoves, filename = "movesAndPositions1.npy"):
 """run a game you stored"""
 testing = np.load(f"data/rawData/{filename}")
 moves = testing[:, 0]
 if (numMoves > len(moves)):
  print("Must enter a lower number of moves than maximum game length. Game length here is: ", len(moves))
  return

 testBoard = chess.Board()

 for i in range(numMoves):
  move = moves[i]
  testBoard.push_san(move)
 return testBoard

In [17]:
def mineGames(numGames : int):
    """mines numGames games of moves"""
    MAX_MOVES = 50 #don't continue games after this number

    for i in range(numGames):
        currentGameMoves = []
        currentGamePositions = []
        board = chess.Board()
        stockfish.set_position([])

        for i in range(MAX_MOVES):
            #randomly choose from those 3 moves
            moves = stockfish.get_top_moves(3)
            #if less than 3 moves available, choose first one, if none available, exit
            if (len(moves) == 0):
                print("game is over")
                break
            elif (len(moves) == 1):
                move = moves[0]["Move"]
            elif (len(moves) == 2):
                move = random.choices(moves, weights=(80, 20), k=1)[0]["Move"]
            else:
                move = random.choices(moves, weights=(80, 15, 5), k=1)[0]["Move"]

            currentGamePositions.append(stockfish.get_fen_position())
            currentGameMoves.append(move) #make sure to add str version of move before changing format
            move = chess.Move.from_uci(str(move)) #convert to format chess package likes
            board.push(move)
            stockfish.set_position(currentGameMoves)
            if (checkEndCondition(board)):
                print("game is over")
                break
        saveData(currentGameMoves, currentGamePositions)

In [19]:
mineGames(5)

Saved successfully
Saved successfully
Saved successfully
Saved successfully
Saved successfully


In [21]:
testBoard = runGame(12, "movesAndPositions5.npy")
testBoard

ValueError: illegal san: 'b1c3' in rnbqkbnr/pp1ppppp/8/2p5/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2